<br><br><br>
<span style="color:red;font-size:60px">Graph Algorithms</span>
<br><br>


<li>breadth-first search</li>
<li>connected components</li>
<li>label propagation</li>
<li>shortest path</li>
<li>triangle count</li>


<br><br><br>
<span style="color:green;font-size:xx-large">Bulk Synchronous Parallel Model</span>
<br><br>

<li>Both GraphX and GraphFrames use the "Bulk Synchronous Parallel" model of processing</li>
<li>BSP model uses 3 supersteps for computation:</li>
<ul>
    <li>Do local computation concurrently for each vertex (or set of vertices)</li>
    <li>Communicate results from one process to another directly (communication and message passing)</li>
    <li>Synchronize activities using <span style="color:red">barrier synchronization</span> (identify barrier tasks that must complete before subsequent processing is possible)</li>
</ul>
<br><br>
<li>Comparison with MapReduce</li>
<ul>
    <li>in-memory state persistence between iterations</li>
    <li>synchronization is restricted to state updates (reduced communication)</li>
    <li>many iterations are possible (good for graphs where iterations may be a factor of the number of vertices) but each iteration is less intensive (since it only deals with updates)</li>
    <li>message passing (between processors) rather than routing through a master node</li>
    <li>each computation in a super step is independent (barrier synchronization ensures this)</li>
</ul>

In [1]:
%%init_spark
launcher.packages= ["graphframes:graphframes:0.8.2-spark3.2-s_2.12"]

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.56.179.196:4043
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1670867547507)
SparkSession available as 'spark'


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._


<img src="social_graph2.png">

In [3]:
val vertexArray = Array(
  (1, "Alice", 28),
  (2, "Bob", 27),
  (3, "Charlie", 65),
  (4, "David", 42),
  (5, "Ed", 55),
  (6, "Fran", 50),
    (7, "Qing",27),
    (8, "Sarika",78),
    (9, "Olafson",17),
    (10, "Birgit",33)
)

val edgeArray = Array(
  (2, 1, 7),
  (1, 2, 13),
  (2, 4, 2),
  (3, 2, 4),
  (3, 6, 3),
  (4, 1, 1),
  (5, 2, 2),
  (5, 3, 8),
  (5, 6, 3),
    (7, 8, 14),
    (7, 9, 2),
    (8, 10, 8),
    (9, 10, 6)
)

val vertex_df = spark.createDataFrame(vertexArray).toDF("id","name","age")
val edge_df = spark.createDataFrame(edgeArray).toDF("src","dst","attr")

val g = GraphFrame(vertex_df, edge_df)

vertexArray: Array[(Int, String, Int)] = Array((1,Alice,28), (2,Bob,27), (3,Charlie,65), (4,David,42), (5,Ed,55), (6,Fran,50), (7,Qing,27), (8,Sarika,78), (9,Olafson,17), (10,Birgit,33))
edgeArray: Array[(Int, Int, Int)] = Array((2,1,7), (1,2,13), (2,4,2), (3,2,4), (3,6,3), (4,1,1), (5,2,2), (5,3,8), (5,6,3), (7,8,14), (7,9,2), (8,10,8), (9,10,6))
vertex_df: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]
edge_df: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
g: org.graphframes.GraphFrame = GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])


<br><br><br>
<span style="color:green;font-size:xx-large">Breadth-first search</span>
<li>Shortest path between two nodes</li>
<li>The shortest path can depend on node attributes</li>
<li>Or we can find shortest paths from multiple nodes to multiple other nodes</li>
<li>bfs computes path lengths based on the number of edges and does not use edge weights</li>

In [ ]:
g.bfs.fromExpr("name='Ed'").toExpr("name='Alice'").run().show
g.bfs.fromExpr("name='Ed'").toExpr("name='Alice'").run().count
g.bfs.fromExpr("age>45").toExpr("age<45").run().show
g.bfs.fromExpr("name='Ed' or name='Bob'").toExpr("name='Alice'").run().show

In [4]:
g.bfs.fromExpr("name='Ed'").toExpr("name='Alice'").run().show

+-----------+---------+------------+---------+--------------+
|       from|       e0|          v1|       e1|            to|
+-----------+---------+------------+---------+--------------+
|{5, Ed, 55}|{5, 2, 2}|{2, Bob, 27}|{2, 1, 7}|{1, Alice, 28}|
+-----------+---------+------------+---------+--------------+



In [5]:
g.bfs.fromExpr("name='Ed'").toExpr("name='Alice'").run().count

res1: Long = 1


In [5]:
g.bfs.fromExpr("age>45").toExpr("age<45").run().show

+----------------+----------+----------------+
|            from|        e0|              to|
+----------------+----------+----------------+
|{3, Charlie, 65}| {3, 2, 4}|    {2, Bob, 27}|
|     {5, Ed, 55}| {5, 2, 2}|    {2, Bob, 27}|
| {8, Sarika, 78}|{8, 10, 8}|{10, Birgit, 33}|
+----------------+----------+----------------+



In [6]:
g.bfs.fromExpr("name='Ed' or name='Bob'").toExpr("name='Alice'").run().show

+------------+---------+--------------+
|        from|       e0|            to|
+------------+---------+--------------+
|{2, Bob, 27}|{2, 1, 7}|{1, Alice, 28}|
+------------+---------+--------------+



<br><br><br>
<span style="color:green;font-size:xx-large">Connected components</span>

<li>GraphFrames connected components function requires a <span style="color:red">checkpoint</span> directory</li>
<li>The algorithm returns a component number for each vertex</li>
<li>The number of distinct component numbers is the number of components of the graph</li>
<li><span style="color:red">connectedComponents</span> returns a list of nodes with along with the component number</li>
<li><span style="color:red">stronglyConnectedComponents</span> returns a list of nodes with along with the component number for each <a href="https://en.wikipedia.org/wiki/Strongly_connected_component">strongly connected component</a></li>

directed graphs, a graph is said to be strongly connected if every vertex is reachable from every other vertex.

In [ ]:
sc.setCheckpointDir("checkpoint")
val cc = g.connectedComponents.run()
cc.show

val result = g.stronglyConnectedComponents.maxIter(10).run()
result.select("id", "component").orderBy("component").show()


In [7]:
sc.setCheckpointDir("checkpoint")
val cc = g.connectedComponents.run()
cc.show

+---+-------+---+---------+
| id|   name|age|component|
+---+-------+---+---------+
|  1|  Alice| 28|        1|
|  2|    Bob| 27|        1|
|  3|Charlie| 65|        1|
|  4|  David| 42|        1|
|  5|     Ed| 55|        1|
|  6|   Fran| 50|        1|
|  7|   Qing| 27|        7|
|  8| Sarika| 78|        7|
|  9|Olafson| 17|        7|
| 10| Birgit| 33|        7|
+---+-------+---+---------+



cc: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [8]:
val result = g.stronglyConnectedComponents.maxIter(10).run()
result.select("id", "component").orderBy("component").show()

+---+---------+
| id|component|
+---+---------+
|  2|        1|
|  1|        1|
|  4|        1|
|  3|        3|
|  5|        5|
|  6|        6|
|  7|        7|
|  8|        8|
|  9|        9|
| 10|       10|
+---+---------+



result: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


<br><br><br>
<span style="color:green;font-size:xx-large">Label propagation</span>
<li>
<li>The label propagation algorithm is a clustering algorithm</li>
<li>Finds "similar" nodes in the graph</li>
<li>the algorithm is iterative but converges very quickly</li>
<li>Roughly:</li>
<ul>
    <li>assign labels randomly to vertices (depending on the size of the graph, this could be to all nodes or just a few)</li>
    <li>update labels based on the frequency of labels in adjacent nodes</li>
    <li>repeat updates</li>
    <li>stop after n iterations</li>
    <li>label propagation is done on the canonical undirected graph</li>
</ul>
<li>Label propagation is used to group nodes in very large graphs - mostly because an exhaustive grouping is computationally infeasible</li>    

In [9]:
val result = g.labelPropagation.maxIter(3).run()
result.select("id", "label").show()


+---+-----+
| id|label|
+---+-----+
|  8|    7|
|  1|    2|
|  9|    7|
| 10|    8|
|  2|    2|
|  3|    2|
|  4|    2|
|  5|    2|
|  6|    2|
|  7|    8|
+---+-----+



result: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


<br><br><br>
<span style="color:green;font-size:xx-large">Shortest path</span>
<li>Compute the shortest path (length) from each vertex to a set of "landmark" vertices </li>
<li>Unfortunately, the shortest path algorithm needs vertex ids to be strings, so we'll convert them to strings!</li>
<li>In the example below, we compute the shortest path from every vertex to vertices 3, 6, and 10</li>
<li>For example, if a company has several factories and several distribution warehouses, it might want to find the shortest path from each factory to each warehouse</li>

In [10]:
val vertexArray = Array(
  (1, "Alice", 28),
  (2, "Bob", 27),
  (3, "Charlie", 65),
  (4, "David", 42),
  (5, "Ed", 55),
  (6, "Fran", 50),
    (7, "Qing",27),
    (8, "Sarika",78),
    (9, "Olafson",17),
    (10, "Birgit",33)
).map(l=>(l._1.toString,l._2,l._3))

val edgeArray = Array(
  (2, 1, 7),
  (1, 2, 13),
  (2, 4, 2),
  (3, 2, 4),
  (3, 6, 3),
  (4, 1, 1),
  (5, 2, 2),
  (5, 3, 8),
  (5, 6, 3),
    (7, 8, 14),
    (7, 9, 2),
    (8, 10, 8),
    (9, 10, 6)
).map(l=>(l._1.toString,l._2.toString,l._3))

val vertex_df = spark.createDataFrame(vertexArray).toDF("id","name","age")
val edge_df = spark.createDataFrame(edgeArray).toDF("src","dst","attr")

val g = GraphFrame(vertex_df, edge_df)

vertexArray: Array[(String, String, Int)] = Array((1,Alice,28), (2,Bob,27), (3,Charlie,65), (4,David,42), (5,Ed,55), (6,Fran,50), (7,Qing,27), (8,Sarika,78), (9,Olafson,17), (10,Birgit,33))
edgeArray: Array[(String, String, Int)] = Array((2,1,7), (1,2,13), (2,4,2), (3,2,4), (3,6,3), (4,1,1), (5,2,2), (5,3,8), (5,6,3), (7,8,14), (7,9,2), (8,10,8), (9,10,6))
vertex_df: org.apache.spark.sql.DataFrame = [id: string, name: string ... 1 more field]
edge_df: org.apache.spark.sql.DataFrame = [src: string, dst: string ... 1 more field]
g: org.graphframes.GraphFrame = GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])


In [11]:
g.shortestPaths.landmarks(Seq("3","6","10")).run().show

+---+-------+---+----------------+
| id|   name|age|       distances|
+---+-------+---+----------------+
|  1|  Alice| 28|              {}|
|  5|     Ed| 55|{6 -> 1, 3 -> 1}|
|  2|    Bob| 27|              {}|
|  8| Sarika| 78|       {10 -> 1}|
|  4|  David| 42|              {}|
|  3|Charlie| 65|{6 -> 1, 3 -> 0}|
|  6|   Fran| 50|        {6 -> 0}|
| 10| Birgit| 33|       {10 -> 0}|
|  9|Olafson| 17|       {10 -> 1}|
|  7|   Qing| 27|       {10 -> 2}|
+---+-------+---+----------------+



<br><br><br>
<span style="color:green;font-size:xx-large">Page Rank</span>
<br><br>
<li>An implementation of Google's page ranking algorithm</li>
<li>Web pages = nodes; links = edges</li>
<li>See <a href="https://en.wikipedia.org/wiki/PageRank">wikipedia</a> for details but the rough idea is:</li>
<ul>
    <li>the rank of a page is higher if it has more incoming links</li>
    <li>the rank of a page is higher if the pages that link to it have higher ranks</li>
</ul>
<li>pagerank takes three arguments</li>
<ul>
    <li><span style="color:red">resetProbability</span>: random walk reset probability (the probability that a page will move to a random page in the network rather than follow a link </li>
    <li><span style="color:red">tol</span>: algorithm stops when it converges to the tol level  </li>
    <li><span style="color:red">maxIter</span>:  stop after the specified number of iterations</li>
</ul>

In [ ]:
val results = g.pageRank.resetProbability(0.15).maxIter(10).run()
// val ranks = g.pageRank.resetProbability(0.15).tol(0.0001).run()
results.vertices.show

val results = g.parallelPersonalizedPageRank.resetProbability(0.01).maxIter(100)
            .sourceIds(Array("1","2")).run()
results.vertices.show(false)

In [12]:
val results = g.pageRank.resetProbability(0.15).maxIter(10).run()
// val ranks = g.pageRank.resetProbability(0.15).tol(0.0001).run()
results.vertices.show

+---+-------+---+-------------------+
| id|   name|age|           pagerank|
+---+-------+---+-------------------+
|  1|  Alice| 28|  2.681863350565778|
|  5|     Ed| 55| 0.2709323459354504|
|  2|    Bob| 27|  2.779774156609249|
|  8| Sarika| 78|0.38607859295801683|
|  4|  David| 42| 1.4539106228273422|
|  3|Charlie| 65|0.34769651061716134|
|  6|   Fran| 50|0.49546752762945484|
| 10| Birgit| 33| 0.9272659539640791|
|  9|Olafson| 17|0.38607859295801683|
|  7|   Qing| 27| 0.2709323459354504|
+---+-------+---+-------------------+



results: org.graphframes.GraphFrame = GraphFrame(v:[id: string, name: string ... 2 more fields], e:[src: string, dst: string ... 2 more fields])


<span style="color:blue;font-size:large">parallelized version of page rank</span>
<li>specify a list of verttices from which to run pagerank in parallel</li>

In [17]:
val results = g.parallelPersonalizedPageRank.resetProbability(0.01).maxIter(100).sourceIds(Array("1","2")).run()
results.vertices.show(false)
// .sourceIds(Array("1","2"))

+---+-------+---+---------------------------------------------------+
|id |name   |age|pageranks                                          |
+---+-------+---+---------------------------------------------------+
|1  |Alice  |28 |(2,[0,1],[0.40321767706296224,0.39655999652854973])|
|5  |Ed     |55 |(2,[0,1],[0.0,0.0])                                |
|2  |Bob    |27 |(2,[0,1],[0.39918550029233296,0.4039384831710437]) |
|8  |Sarika |78 |(2,[0,1],[0.0,0.0])                                |
|4  |David  |42 |(2,[0,1],[0.19759682264470482,0.19950152030040658])|
|3  |Charlie|65 |(2,[0,1],[0.0,0.0])                                |
|6  |Fran   |50 |(2,[0,1],[0.0,0.0])                                |
|10 |Birgit |33 |(2,[0,1],[0.0,0.0])                                |
|9  |Olafson|17 |(2,[0,1],[0.0,0.0])                                |
|7  |Qing   |27 |(2,[0,1],[0.0,0.0])                                |
+---+-------+---+---------------------------------------------------+



results: org.graphframes.GraphFrame = GraphFrame(v:[id: string, name: string ... 2 more fields], e:[src: string, dst: string ... 2 more fields])


<br><br><br>
<span style="color:green;font-size:xx-large">triangle count</span>
<li>the number of triangles that each vertice belongs to</li>
<li>for example, Bob belongs to two triangles: (Bob, David, Alice) and (Bob, Charlie, Ed)</li>
<li>triangles assume an undirected graph</li>

In [6]:
val results = g.triangleCount.run()
results.show

+-----+---+-------+---+
|count| id|   name|age|
+-----+---+-------+---+
|    1|  1|  Alice| 28|
|    2|  2|    Bob| 27|
|    2|  3|Charlie| 65|
|    2|  5|     Ed| 55|
|    1|  4|  David| 42|
|    1|  6|   Fran| 50|
|    0|  7|   Qing| 27|
|    0|  8| Sarika| 78|
|    0|  9|Olafson| 17|
|    0| 10| Birgit| 33|
+-----+---+-------+---+



results: org.apache.spark.sql.DataFrame = [count: bigint, id: int ... 2 more fields]


<br><br>
<span style="color:green;font-size:xx-large">Clustering coefficient using triangle count</span>
<li>clustering coefficient = number of triangles a vertex belongs to divided by the number of possible triangles</li>
<li>for example, Alice belongs to 1 triangle (Alice, Bob, David) and, since she has only two adjacent vertices, the number of possible triangles is also 1. Alice's clustering coefficient is 1/1 = 1.0</li>
<li>Bob belongs to two triangles. The possible triangles are: 
    <ul>
        <li>(Bob, David, Alice)</li>
        <li>(Bob, David, Charlie)</li>
        <li>(Bob, Alice, Charlie)</li>
        <li>(Bob, David, Ed)</li>
        <li>(Bob, Alice, Ed)</li>
        <li>(Bob, Charlie, Ed)</li>
    </ul>
<li>thus, Bob's clustering coefficient is 2/6 = 0.33</li>


In [18]:
//Copied from previous notebook
def make_undirected_graph(g: GraphFrame) = {
    val u_edge_df = g.find("(a)-[]->(b)")
        .select($"a.id".as("src"),$"b.id".as("dst"))
        .withColumn("swap",when(col("src")<col("dst"),col("dst")))
        .withColumn("dst",
                    when(col("swap").isNotNull,col("src"))
                    .otherwise(col("dst")))
        .withColumn("src",
                    when(col("swap").isNotNull,col("swap"))
                   .otherwise(col("src")))
        .drop(col("swap"))
        .distinct
    val u_vertices_df = g.vertices
    val u_g = GraphFrame(u_vertices_df,u_edge_df)    
    u_g
}

make_undirected_graph: (g: org.graphframes.GraphFrame)org.graphframes.GraphFrame


In [19]:
val triangles = g.triangleCount.run().withColumnRenamed("id","t_id") 
//Get the number of triangles each vertex belongs to
val degrees = make_undirected_graph(g).degrees 
//Get the number of adjacent vertices for each vertex
val possible = degrees.withColumn("possible",col("degree")*(col("degree")-1)/lit(2)) 
//Calculate possible triangles
val joined = triangles.select("t_id","count").join(possible,triangles("t_id")===possible("id")) 
val coeff = joined.withColumn("coeff",col("count")/col("possible"))
coeff.select("id","coeff").show

+---+------------------+
| id|             coeff|
+---+------------------+
|  1|               1.0|
|  2|0.3333333333333333|
|  3|0.6666666666666666|
|  5|0.6666666666666666|
|  4|               1.0|
|  6|               1.0|
|  7|               0.0|
|  8|               0.0|
|  9|               0.0|
| 10|               0.0|
+---+------------------+



triangles: org.apache.spark.sql.DataFrame = [count: bigint, t_id: string ... 2 more fields]
degrees: org.apache.spark.sql.DataFrame = [id: string, degree: int]
possible: org.apache.spark.sql.DataFrame = [id: string, degree: int ... 1 more field]
joined: org.apache.spark.sql.DataFrame = [t_id: string, count: bigint ... 3 more fields]
coeff: org.apache.spark.sql.DataFrame = [t_id: string, count: bigint ... 4 more fields]


In [20]:
degrees.withColumn("possible",col("degree")*(col("degree")-1)/lit(2)).show

+---+------+--------+
| id|degree|possible|
+---+------+--------+
|  7|     2|     1.0|
|  3|     3|     3.0|
|  8|     2|     1.0|
|  5|     3|     3.0|
|  6|     2|     1.0|
|  9|     2|     1.0|
|  1|     2|     1.0|
| 10|     2|     1.0|
|  4|     2|     1.0|
|  2|     4|     6.0|
+---+------+--------+



In [21]:
g.triangleCount.run().withColumnRenamed("id","t_id").show

+-----+----+-------+---+
|count|t_id|   name|age|
+-----+----+-------+---+
|    1|   1|  Alice| 28|
|    2|   2|    Bob| 27|
|    2|   3|Charlie| 65|
|    2|   5|     Ed| 55|
|    1|   4|  David| 42|
|    1|   6|   Fran| 50|
|    0|   7|   Qing| 27|
|    0|   8| Sarika| 78|
|    0|   9|Olafson| 17|
|    0|  10| Birgit| 33|
+-----+----+-------+---+



<br><br><br>
<span style="color:green;font-size:xx-large">aggregateMessages</span>

<li>Neighborhood aggregation function through messaging</li>
<li>Messages</li>
<ul>
    <li>source data (e.g., AggregateMessages.src("age"))</li>
    <li>destination data (e.g., AggregateMessages.dst("age))</li>
    <li>edge data (e.g., AggregateMessages.edge("attr")</li> 
    <li>A message is sent from each vertex either from a src to a dst (sendToDst) or from a dst to a source (sendToSrc)</li>
    <li>A function then processes the received message (agg)</li>
</ul>

In [ ]:
import org.apache.spark.sql.functions
import org.graphframes.lib.AggregateMessages
g.aggregateMessages
    .sendToDst(AggregateMessages.edge("attr")) 
    //Send the edge attr to value to the destination
    .agg(sum(AggregateMessages.msg).as("alllikes")).show 
    //Aggregate messages by summing them up

In [5]:
import org.apache.spark.sql.functions
import org.graphframes.lib.AggregateMessages

import org.apache.spark.sql.functions
import org.graphframes.lib.AggregateMessages


<span style="color:blue;font-size:large">Calculate total incoming likes for every node</span>
<li>For example, Bob has 3 incoming edges, each with attr values 13, 2, 4</li>


In [6]:
g.aggregateMessages
    .sendToDst(AggregateMessages.edge("attr")) //Send the edge attr to value to the destination
    .agg(sum(AggregateMessages.msg).as("alllikes")).show //Aggregate messages by summing them up

+---+--------+
| id|alllikes|
+---+--------+
|  1|       8|
|  4|       2|
|  2|      19|
|  6|       6|
|  3|       8|
|  9|       2|
|  8|      14|
| 10|      14|
+---+--------+



<span style="color:blue;font-size:large">Try this: Calculate total outgoing likes for each person</span>


In [7]:
g.aggregateMessages
    .sendToSrc(AggregateMessages.edge("attr")) //Send the edge attr to value to the destination
    .agg(sum(AggregateMessages.msg).as("alllikes")) //Aggregate messages by summing them up
    .show

+---+--------+
| id|alllikes|
+---+--------+
|  2|       9|
|  1|      13|
|  3|       7|
|  4|       1|
|  5|      13|
|  7|      16|
|  9|       6|
|  8|       8|
+---+--------+



<span style="color:blue;font-size:large">for each person, who likes them the most and how much?</span>
<li>This is going to be embarrassingly complicated!</li>

In [8]:
val max_df = g.aggregateMessages
    .sendToDst(AggregateMessages.edge("attr"))
    .agg(max(AggregateMessages.msg))
    .withColumnRenamed("max(MSG)","maxval")

max_df: org.apache.spark.sql.DataFrame = [id: int, maxval: int]


In [9]:
max_df.show

+---+------+
| id|maxval|
+---+------+
|  1|     7|
|  4|     2|
|  2|    13|
|  6|     3|
|  3|     8|
|  9|     2|
|  8|    14|
| 10|     8|
+---+------+



In [15]:
max_df.printSchema

root
 |-- id: integer (nullable = false)
 |-- maxval: integer (nullable = true)



In [16]:
max_df.createOrReplaceTempView("max_db")

In [17]:
spark.sql("select max(maxval) from max_db").show

+-----------+
|max(maxval)|
+-----------+
|         14|
+-----------+



In [18]:
spark.sql("select id, maxval from max_db where maxval = (select max(maxval) from max_db)").show


+---+------+
| id|maxval|
+---+------+
|  8|    14|
+---+------+

